# CQW - Dashboard Reports
Updated: 2019-10-16

These dashboard reports were generated by Cognos. 

  - Report dwq01 - Sales by Category and Classification
  - Report dwq02 - Weekly Sales by Category By Location
  - Report dwq03 - Store Net Profit Margin
  - Report dwq04 - Net Profit Margin by Store by 4th Quarter
  - Report dwq05 - Gross Profit Across All Stores
  - Report dwq06 - Sales of Electonics Across All Stores
  - Report dwq07 - Sales of Electronics at Individual Stores
  - Report dwq08 - Weekly Electronic Sales
  - Report dwq09 - Gross Store Profit in 4Q of 2001 and 2002
  - Report dwq10 - Total Store Revenue
  - Report dwq11 - Total Store Revenue in 4Q of 2001 and 2002
  - Report dwq12 - Sales by Month in 3Q/4Q of 2002
  - Report dwq13 - Wholesale Cost of Electronics by Month in 2002


### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

### Create a Function for Running the Queries
This function is used to time the queries and return the total time taken. The parameter is the SQL that will be run and the function will execute against the row-based and col-based tables. 

In [ ]:
def runquery(sql):
    
    %sql -q SET CURRENT SCHEMA = CQWROW
        
    print("Starting execution of ROW query")    
    start_time = time.time()

    %sql -q {sql}
    
    end_time = time.time()
    row = end_time - start_time
    
    print("Row run time = {0}".format(row))
    
    %sql -q SET CURRENT SCHEMA = CQWCOL
    
    print()
    print("Starting execution of COLUMNAR query")        
    
    start_time = time.time()

    results = %sql -q {query}
    
    end_time = time.time()
    col = end_time - start_time
    
    print("Column run time = {0}".format(col))    
    
    pdisplay(results)
    
    %sql -bar VALUES ('Row',:row),('Column',:col)

## Sales by Category and Classification

In [ ]:
query = '''
SELECT
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    "ITEM"."I_CLASS" AS "Items_Trimmed_Clasification", 
    SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "Quantity__Store_Sales_", 
    SUM("STORE_SALES"."SS_NET_PROFIT") * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "SubTotal_Net_Profit__Store_Sales_", 
    SUM("STORE_SALES"."SS_NET_PAID") * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "SubTotal_Revenue__Store_Sales_"
FROM
    "ITEM" "ITEM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
WHERE 
    "ITEM"."I_CATEGORY" IN ( 
        'Electronics' ) AND
    "ITEM"."I_CLASS" IN ( 
        'memory', 
        'televisions', 
        'camcorders', 
        'portable', 
        'wireless' ) 
GROUP BY 
    "ITEM"."I_CATEGORY", 
    "ITEM"."I_CLASS"
'''
runquery(query)

## Weekly Sales by Category By Location

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_WEEK_SEQ" AS "Date_D_WEEK_SEQ", 
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    "STORE"."S_COUNTRY" AS "Stores_S_COUNTRY", 
    "STORE"."S_STATE" AS "Stores_S_STATE", 
    "STORE"."S_COUNTY" AS "Stores_S_COUNTY", 
    "STORE"."S_CITY" AS "Stores_S_CITY", 
    SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "WholeSale_Cost__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK"
            INNER JOIN "ITEM" "ITEM"
            ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK"
                INNER JOIN "STORE" "STORE"
                ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2002 ) AND
    "DATE_DIM"."D_QOY" IN ( 
        4 ) AND
    "DATE_DIM"."D_MOY" IN ( 
        12 ) AND
    "ITEM"."I_CATEGORY" IN ( 
        'Electronics' ) AND
    "STORE"."S_COUNTRY" IN ( 
        'United States' ) 
GROUP BY 
    "DATE_DIM"."D_WEEK_SEQ", 
    "ITEM"."I_CATEGORY", 
    "STORE"."S_COUNTRY", 
    "STORE"."S_STATE", 
    "STORE"."S_COUNTY", 
    "STORE"."S_CITY"
'''
runquery(query)

## Store Net Profit Margin

In [ ]:
query = '''
SELECT
    CASE 
        WHEN SUM("STORE_SALES"."SS_NET_PAID") = 0 THEN 0
        ELSE CAST(SUM("STORE_SALES"."SS_NET_PROFIT") AS DOUBLE PRECISION) / NULLIF(SUM("STORE_SALES"."SS_NET_PAID"), 0)
    END AS "Net_Profit_Margin__Store_Sales_"
FROM
    "STORE_SALES"
'''
runquery(query)

## Net Profit Margin by Store by 4th Quarter

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    "DATE_DIM"."D_QOY" AS "Date_D_QOY", 
    CASE 
        WHEN SUM("STORE_SALES"."SS_NET_PAID") = 0 THEN 0
        ELSE CAST(SUM("STORE_SALES"."SS_NET_PROFIT") AS DOUBLE PRECISION) / NULLIF(SUM("STORE_SALES"."SS_NET_PAID"), 0)
    END AS "Net_Profit_Margin__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2001, 
        2002 ) AND
    "DATE_DIM"."D_QOY" IN ( 
        4 ) 
GROUP BY 
    "DATE_DIM"."D_YEAR", 
    "DATE_DIM"."D_QOY"
'''
runquery(query)

## Gross Profit Across All Stores

In [ ]:
query = '''
SELECT
    (SUM("STORE_SALES"."SS_NET_PAID") - SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2)))) * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "SubTotal_Gross_Profit__Store_Sales_"
FROM
    "STORE_SALES" "STORE_SALES"
'''
runquery(query)

## Sales of Electonics Across All Stores

In [ ]:
query = '''
SELECT
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "WholeSale_Cost__Store_Sales_"
FROM
    "ITEM" "ITEM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
WHERE 
    "ITEM"."I_CATEGORY" IN ( 
        'Electronics' ) 
GROUP BY 
    "ITEM"."I_CATEGORY"
'''
runquery(query)

## Sales of Electronics at Individual Stores

In [ ]:
query = '''
SELECT
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    "STORE"."S_COUNTRY" AS "Stores_S_COUNTRY", 
    "STORE"."S_STATE" AS "Stores_S_STATE", 
    "STORE"."S_COUNTY" AS "Stores_S_COUNTY", 
    "STORE"."S_CITY" AS "Stores_S_CITY", 
    SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "WholeSale_Cost__Store_Sales_"
FROM
    "ITEM" "ITEM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK"
            INNER JOIN "STORE" "STORE"
            ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "ITEM"."I_CATEGORY" IN ( 
        'Electronics' ) AND
    "STORE"."S_COUNTRY" IN ( 
        'United States' ) 
GROUP BY 
    "ITEM"."I_CATEGORY", 
    "STORE"."S_COUNTRY", 
    "STORE"."S_STATE", 
    "STORE"."S_COUNTY", 
    "STORE"."S_CITY"
'''
runquery(query)

## Weekly Electronic Sales

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_WEEK_SEQ" AS "Date_D_WEEK_SEQ", 
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "WholeSale_Cost__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK"
            INNER JOIN "ITEM" "ITEM"
            ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2002 ) AND
    "DATE_DIM"."D_QOY" IN ( 
        4 ) AND
    "DATE_DIM"."D_MOY" IN ( 
        12 ) AND
    "ITEM"."I_CATEGORY" IN ( 
        'Electronics' ) 
GROUP BY 
    "DATE_DIM"."D_WEEK_SEQ", 
    "ITEM"."I_CATEGORY"
'''
runquery(query)

## Gross Store Profit in 4Q of 2001 and 2002

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    "DATE_DIM"."D_QOY" AS "Date_D_QOY", 
    (SUM("STORE_SALES"."SS_NET_PAID") - SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2)))) * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "SubTotal_Gross_Profit__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2001, 
        2002 ) AND
    "DATE_DIM"."D_QOY" IN ( 
        4 ) 
GROUP BY 
    "DATE_DIM"."D_YEAR", 
    "DATE_DIM"."D_QOY"
'''
runquery(query)

## Total Store Revenue

In [ ]:
query = '''
SELECT
    SUM("STORE_SALES"."SS_NET_PAID") * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "SubTotal_Revenue__Store_Sales_"
FROM
    "STORE_SALES" 
'''
runquery(query)

## Total Store Revenue in 4Q of 2001 and 2002

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    "DATE_DIM"."D_QOY" AS "Date_D_QOY", 
    SUM("STORE_SALES"."SS_NET_PAID") * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "SubTotal_Revenue__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2001, 
        2002 ) AND
    "DATE_DIM"."D_QOY" IN ( 
        4 ) 
GROUP BY 
    "DATE_DIM"."D_YEAR", 
    "DATE_DIM"."D_QOY"
'''
runquery(query)

## Sales by Month in 3Q/4Q of 2002

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    "DATE_DIM"."D_QOY" AS "Date_D_QOY", 
    "DATE_DIM"."D_MOY" AS "Date_D_MOY", 
    SUM("STORE_SALES"."SS_NET_PAID") AS "Net_Paid__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2002 ) AND
    "DATE_DIM"."D_QOY" IN ( 
        4, 
        3 ) 
GROUP BY 
    "DATE_DIM"."D_YEAR", 
    "DATE_DIM"."D_QOY", 
    "DATE_DIM"."D_MOY"
'''
runquery(query)

## Wholesale Cost of Electronics by Month in 2002

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    "DATE_DIM"."D_QOY" AS "Date_D_QOY", 
    "DATE_DIM"."D_MOY" AS "Date_D_MOY", 
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "WholeSale_Cost__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK"
            INNER JOIN "ITEM" "ITEM"
            ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2002 ) AND
    "DATE_DIM"."D_QOY" IN ( 
        4, 
        3 ) AND
    "ITEM"."I_CATEGORY" IN ( 
        'Electronics' ) 
GROUP BY 
    "DATE_DIM"."D_YEAR", 
    "DATE_DIM"."D_QOY", 
    "DATE_DIM"."D_MOY", 
    "ITEM"."I_CATEGORY"
'''
runquery(query)

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]